In [39]:
import pandas as pd
import requests
import json
import sqlite3

In [40]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 55.7522,
	"longitude": 37.6156,
	"daily": ["temperature_2m_max", "temperature_2m_min", "sunrise", "sunset", "rain_sum", "snowfall_sum"],
	"timezone": "Europe/Moscow",
	"forecast_days": 1
}

In [41]:
response = requests.get(url, params=params)

In [42]:
data = json.loads(response.text)

In [43]:
column_name = list(data['daily'].keys())

In [44]:
df = pd.json_normalize(data['daily'])

In [45]:
exploded_df = df.explode(['time',
 'temperature_2m_max',
 'temperature_2m_min',
 'sunrise',
 'sunset',
 'rain_sum',
 'snowfall_sum'], ignore_index = True)

In [46]:
exploded_df.rename(columns=
{
          'time' : 'date',
          'temperature_2m_max' : 'temperature_max',
          'temperature_2m_min' : 'temperature_min',
          'rain_sum' : 'rain',
          'snowfall_sum' : 'snowfall'
},
inplace=True)

In [47]:
exploded_df.insert(loc= 1 , column='location', value='Moscow')

In [48]:
con_sqlite = sqlite3.connect('new_db_weather')

In [49]:
exploded_df.to_sql(con=con_sqlite, name = 'weather', index=False, if_exists='append')

1